### Data
- S&P 500 companies list https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks
- S&P 500 stock prices yfinance library

### Stocks

#### Get stock prices history for the whole period till now

In [46]:
import pandas as pd
import yfinance as yf
import time
import os
import datetime

tickers_log = pd.read_csv('news/constituents.csv')

tickers_log = tickers_log['Symbol'].str.lower().sort_values().tolist()
#  create dir for current_date
current_date = datetime.datetime.now().date().strftime("%Y%m%d")
os.makedirs(f'data/{current_date}', exist_ok=True)
for t in tickers_log:
    ticker = yf.Ticker(t)
    h = ticker.history(period="max")
    h.to_csv(f'data/{current_date}/' + t + ".csv", sep=",")
    print("Done", t)
    time.sleep(.5)

Done voo


#### Fill table stocks in stocks database with data from yfinance library output

In [ ]:
import pandas as pd
import sqlite3, os
conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS stocks")
c.execute("DROP INDEX IF EXISTS idx_unique_stocks_ticker_date")
c.execute("CREATE TABLE EXISTS stocks (date text, ticker text, open real, high real, low real, close real, volume real, dividends real, stocks_split real) STRICT")
c.execute('CREATE UNIQUE INDEX idx_unique_stocks_ticker_date ON stocks(ticker,date)')
# write to database data from csv
df = pd.read_csv('./news/constituents.csv')
for index, row in df.iterrows():
    ticker = row['Symbol'].lower()
    fname = './data/20241209/{}.csv'.format(ticker)
    if not os.path.exists(fname):
        print('File {} not found'.format(fname))
        continue
    df = pd.read_csv(fname)
    for index, row in df.iterrows():
        date = row['Date'][:10]
        open = row['Open']
        high = row['High']
        low = row['Low']
        close = row['Close']
        volume = row['Volume']
        dividends = row['Dividends']
        stocks_split = row['Stock Splits']
        c.execute("INSERT INTO stocks (date, ticker, open, high, low, close, volume, dividends, stocks_split) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                  (date, ticker, open, high, low, close, volume, dividends, stocks_split))
c.close()
conn.commit()
conn.close()

#### Get latest data from yfinance library and update table stocks in stocks database



In [ ]:
import yfinance as yf
import pandas as pd
import sqlite3, os
from datetime import datetime, timedelta

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute('SELECT DISTINCT(ticker) FROM stocks ORDER BY ticker')
tickers_log = c.fetchall()

for ticker in tickers_log:
    ticker = ticker[0]
    print('Processing ticker {}'.format(ticker))
    c.execute('SELECT max(date) FROM stocks WHERE ticker = ?', (ticker,))
    max_date = c.fetchone()[0]
    if max_date is None:
        print('Need to get the whole history for this ticker {}'.format(ticker))
        yahoo_ticker = yf.Ticker(ticker)
        data = yahoo_ticker.history(period="max")
        for idx, row in data.iterrows():
            date = idx.date()
            open = row['Open']
            high = row['High']
            low = row['Low']
            close = row['Close']
            volume = row['Volume']
            dividends = row['Dividends']
            stocks_split = row['Stock Splits']
            c.execute("INSERT INTO stocks (date, ticker, open, high, low, close, volume, dividends, stocks_split) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                    (date.strftime("%Y-%m-%d"), ticker, open, high, low, close, volume, dividends, stocks_split))
        data.to_csv(f'data/yahoo_finance/' + ticker + ".csv", sep=",")
        continue

    max_date = datetime.strptime(max_date[:10], '%Y-%m-%d')
    day_after = max_date + timedelta(days=1)
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    if day_after >= yesterday:
        print('No need to update ticker {}'.format(ticker))
        continue
    yahoo_ticker = yf.Ticker(ticker)
    data = yahoo_ticker.history(start=day_after.strftime("%Y-%m-%d") , end=yesterday.strftime("%Y-%m-%d"))
    
    data.to_csv(f'data/yahoo_finance/' + ticker + ".csv", sep=",")
    for idx, row in data.iterrows():
        date = idx.date()
        if pd.notna(row['Open']) == False or pd.notna(row['High']) == False or pd.notna(row['Low']) == False or pd.notna(row['Close']) == False or pd.notna(row['Volume']) == False:
            print('Skipping data for date {}'.format(date))
            continue
        open = row['Open']
        high = row['High']
        low = row['Low']
        close = row['Close']
        volume = row['Volume']
        dividends = row['Dividends']
        stocks_split = row['Stock Splits']
        c.execute("INSERT INTO stocks (date, ticker, open, high, low, close, volume, dividends, stocks_split) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
            (date.strftime('%Y-%m-%d'), ticker, open, high, low, close, volume, dividends, stocks_split))
c.close()
conn.commit()
conn.close()


#### Get diff between companies in S&P 500 list and companies in stocks table and add them to stocks table

In [4]:
import pandas as pd
import sqlite3, os
from datetime import datetime, timedelta
import yfinance as yf

# get all tickers from file
df = pd.read_csv('./news/constituents.csv')

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute('SELECT DISTINCT(ticker) FROM stocks ORDER BY ticker')
tickers_log = c.fetchall()
for index, row in df.iterrows():
    ticker_file = row['Symbol'].lower()
    found = False
    for ticker in tickers_log:
        ticker_db = ticker[0]
        if ticker_file == ticker_db:
            found = True
            break
    if found == False:
        # TODO: make it a function
        print('Adding new ticker {}'.format(ticker_file))
        yahoo_ticker = yf.Ticker(ticker_file)
        data = yahoo_ticker.history(period="max")
        for idx, row in data.iterrows():
            date = idx.date()
            open = row['Open']
            high = row['High']
            low = row['Low']
            close = row['Close']
            volume = row['Volume']
            dividends = row['Dividends']
            stocks_split = row['Stock Splits']
            c.execute("INSERT INTO stocks (date, ticker, open, high, low, close, volume, dividends, stocks_split) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)", 
                    (date.strftime("%Y-%m-%d"), ticker_file, open, high, low, close, volume, dividends, stocks_split))
        data.to_csv(f'data/yahoo_finance/' + ticker_file + ".csv", sep=",")

c.close()
conn.commit()
conn.close()



### Helper functions

#### Execute queries with retries

In [3]:
import sqlite3
import time

def execute_with_retry(conn, query, params=(), retries=5, delay=1):
	for i in range(retries):
		try:
			c = conn.cursor()
			c.execute(query, params)
			conn.commit()
			return
		except sqlite3.OperationalError as e:
			if 'database is locked' in str(e):
				time.sleep(delay)
			else:
				raise
		finally:
			c.close()
	raise Exception("Failed to execute query after multiple retries")

#### Get all dates of particular day of week in specific date range that are present in stocks table



In [4]:
import sqlite3
from datetime import datetime, timedelta

def get_dates_of_day_of_week_in_range(start_date_str, end_date_str, day_of_week):
    # get all days of day_of_week for period from start_date to end_date from calendar
    dates = []
    try:
        current_date = datetime.strptime(start_date_str, "%Y-%m-%d")
        end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    except Exception as e:
        raise ValueError("Invalid date format", e)
    if day_of_week < 0 or day_of_week > 6:
        raise ValueError("Invalid day of week", day_of_week)

    while current_date <= end_date:
        if current_date.weekday() == day_of_week:
            dates.append(current_date)        
        current_date += timedelta(days=1)

    return dates    

def filter_dates_not_in_stock_table(dates):
    conn = sqlite3.connect('data/stocks.db', timeout=30)
    dates_in_range = []
    for date in dates:
        c = conn.cursor()
        c.execute('SELECT date FROM stocks WHERE date = ? LIMIT 1', (date.strftime("%Y-%m-%d"),))
        if c.fetchone():
            dates_in_range.append(date)
        c.close()    
    return dates_in_range

In [ ]:
import unittest
from datetime import datetime, timedelta

class TestGetDatesOfDayOfWeek(unittest.TestCase):
    def test_one_week_range(self):
        # Test for a single week range
        result = get_dates_of_day_of_week_in_range("2024-01-01", "2024-01-07", 0)  # Monday
        self.assertEqual(len(result), 1)
        self.assertEqual(result[0].strftime("%Y-%m-%d"), "2024-01-01")

    def test_month_range(self):
        # Test for a month range
        result = get_dates_of_day_of_week_in_range("2024-01-01", "2024-01-31", 2)  # Wednesday
        self.assertEqual(len(result), 5)  # Should have 5 Wednesdays in January 2024
        self.assertTrue(all(d.weekday() == 2 for d in result))

    def test_empty_range(self):
        # Test when end date is before start date
        result = get_dates_of_day_of_week_in_range("2024-01-31", "2024-01-01", 1)
        self.assertEqual(len(result), 0)

    def test_same_day(self):
        # Test when start and end date are the same
        result = get_dates_of_day_of_week_in_range("2024-01-01", "2024-01-01", 0)  # Monday
        self.assertEqual(len(result), 1)
        result = get_dates_of_day_of_week_in_range("2024-01-01", "2024-01-01", 1)  # Tuesday
        self.assertEqual(len(result), 0)

    def test_invalid_day_of_week(self):
        # Test with invalid day of week values
        with self.assertRaises(ValueError):
            get_dates_of_day_of_week_in_range("2024-01-01", "2024-01-07", 7)
        with self.assertRaises(ValueError):
            get_dates_of_day_of_week_in_range("2024-01-01", "2024-01-07", -1)

    def test_invalid_date_format(self):
        # Test with invalid date format
        with self.assertRaises(ValueError):
            get_dates_of_day_of_week_in_range("2024/01/01", "2024-01-07", 0)
        with self.assertRaises(ValueError):
            get_dates_of_day_of_week_in_range("2024-01-01", "invalid_date", 0)

    def test_leap_year(self):
        # Test handling of leap year
        result = get_dates_of_day_of_week_in_range("2024-02-01", "2024-02-29", 3)  # Thursday
        self.assertEqual(len(result), 5)
        self.assertTrue(all(d.weekday() == 3 for d in result))

    def test_all_days_in_week(self):
        # Test for all days in a week
        start_date = "2024-01-01"
        end_date = "2024-01-07"
        total_days = sum(len(get_dates_of_day_of_week_in_range(start_date, end_date, day)) 
                        for day in range(7))
        self.assertEqual(total_days, 7)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

In [ ]:
import sqlite3

conn = sqlite3.connect('bigdata/one/logs.db')
c = conn.cursor()
# get unique indicators names
c.execute('SELECT distinct(ticker) FROM log_models')
fetch = c.fetchall()
tickers_log = [ticker[0] for ticker in fetch]

print("Total number of tickers: ", len(tickers_log))
c.close()
conn.close()

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute('SELECT distinct(ticker) FROM stocks')
fetch = c.fetchall()
tickers_stocks = [ticker[0] for ticker in fetch]
print("Total number of tickers: ", len(tickers_stocks))
c.close()
conn.close()

for ticker in tickers_stocks:
    if ticker not in tickers_log:
        print("{}".format(ticker))


### News

#### Reset news table

In [31]:
import sqlite3

conn = sqlite3.connect('data/stocks.db', timeout=30)
try:
	execute_with_retry(conn, "DROP TABLE IF EXISTS news")
	execute_with_retry(conn, "VACUUM")
	execute_with_retry(conn, "CREATE TABLE EXISTS news (date text, ticker text, json_news text, avg_sentiment real) STRICT")
	execute_with_retry(conn, 'CREATE UNIQUE INDEX idx_unique_news_ticker_date ON news(ticker,date)')
finally:
	if conn:
		conn.close()

#### Average news sentiment for the company

In [38]:
def average_news_sentiment(ticker, news) -> float:
    try:
        if news['items'] == 0:
            return 0.0
        if len(news['feed']) == 0:
            return 0.0
    except Exception as e:
        print("Ticker",ticker,"News json", news)
        return 0.0    
    scores_arr = []
    for item in news['feed']:
        for sentiment in item['ticker_sentiment']:
            if sentiment['ticker'].lower() == ticker.lower():
                scores_arr.append(float(sentiment['ticker_sentiment_score'])*float(sentiment['relevance_score']))
    if len(scores_arr) == 0:
        return 0.0
    return sum(scores_arr)/len(scores_arr)

#### Fill up news table with news from news/data/{company}/{date}.csv

In [ ]:
import sqlite3, os
from datetime import datetime
import json

conn = sqlite3.connect('data/stocks.db')
for company_news_dir in os.listdir('./news/data/'):
    if os.path.isdir('./news/data/' + company_news_dir) == False:
        continue
    for model_name in os.listdir('./news/data/' + company_news_dir):
        if model_name.endswith('.json') == False:
            continue
        try:
            with open('./news/data/'+company_news_dir+'/' + model_name, 'r') as file:
                json_data = json.load(file)
                if 'items' in json_data:
                    if json_data['items'] == '0':
                        file.close()
                        continue
                file.close()
        except Exception as e:
            print('Error reading file {}, error: {}'.format(model_name, e))
            continue
        
        ticker = company_news_dir
        date = datetime.strptime(model_name[:8], "%Y%m%d")
        avg_sentiment = average_news_sentiment(ticker, json_data)
        json_news = json.dumps(json_data)
        if avg_sentiment == 0.0:
            continue
        execute_with_retry(conn, "INSERT OR REPLACE INTO news (date, ticker, json_news, avg_sentiment) VALUES (?, ?, ?, ?)",
                    (date.strftime("%Y-%m-%d"), ticker, json_news, avg_sentiment), retries=2, delay=1)        
conn.commit()
conn.close()


#### Clean up invalid news files


In [ ]:
import os
removed_files = 0
bad_arr = []
for company_news_dir in os.listdir('./news/data/'):
    if os.path.isdir('./news/data/' + company_news_dir) == False:
        continue
    amount_of_bad_files = 0
    for model_name in os.listdir('./news/data/' + company_news_dir):
        try:
            with open('./news/data/' + company_news_dir + '/' + model_name, 'r') as file:
                json_data = json.load(file)
                if 'items' not in json_data:
                    os.remove('./news/data/' + company_news_dir + '/' + filename)
                    removed_files += 1
                    amount_of_bad_files += 1
                    continue
        except Exception as e:
            print('Error reading file {}, error: {}'.format(model_name, e))
            continue
    if amount_of_bad_files > 0:
        bad_arr.append((company_news_dir, amount_of_bad_files))
print(f'Removed {removed_files} files')
print(bad_arr)

#### Get records with zero news sentiment (expected to be empty) 

In [ ]:
import pandas as pd
import sqlite3, os
from datetime import datetime, timedelta

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute("SELECT ticker, date FROM news where avg_sentiment = 0.0")
data = c.fetchall()

print("Tickers with 0.0 sentiment")
for row in data:
    ticker = row[0]
    date = row[1]
    print("Ticker {} for date {}".format(ticker, date))
c.close()
conn.close()

#### Get latest news from news files and update news table

In [ ]:
import sqlite3, os
from datetime import datetime, timedelta
import json

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute('SELECT DISTINCT(ticker) FROM news ORDER BY ticker')
tickers_log = c.fetchall()
c.close()

for ticker in tickers_log:
    ticker = ticker[0]
    # print('Processing ticker {}'.format(ticker))
    c = conn.cursor()
    c.execute('SELECT max(date) FROM news WHERE ticker = ?', (ticker,))
    max_date = c.fetchone()[0]
    c.close()
    if max_date is None:
        print('Need to get the whole history for this ticker {}'.format(ticker))
        continue
    max_date = datetime.strptime(max_date[:10], '%Y-%m-%d')
    day_after = max_date + timedelta(days=1)
    today = datetime.now()
    yesterday = today - timedelta(days=1)
    if day_after > yesterday:
        # print('No need to update ticker {}'.format(ticker))
        continue
    for company_news_dir in os.listdir('./news/data/'):
        if company_news_dir != ticker:
            continue
        if os.path.isdir('./news/data/' + company_news_dir) == False:
            continue
        
        for model_name in os.listdir('./news/data/' + company_news_dir):
            if model_name.endswith('.json') == False:
                continue
            date = datetime.strptime(model_name[:8], "%Y%m%d")
            if date <= max_date:
                continue
            with open('./news/data/'+company_news_dir+'/' + model_name, 'r') as file:
                json_data = json.load(file)
                if 'items' in json_data:
                    if json_data['items'] == '0':
                        file.close()
                        continue
                file.close()
            json_news = json.dumps(json_data)
            print('Adding news for {} for date {}'.format(ticker, date.strftime('%Y-%m-%d')))
            avg_sentiment = average_news_sentiment(ticker, json_data)
            execute_with_retry(conn, "INSERT INTO news (date, ticker, json_news, avg_sentiment) VALUES (?, ?, ?, ?)",
                        (date.strftime("%Y-%m-%d"), ticker, json_news, avg_sentiment), retries=2, delay=1)
conn.commit()
conn.close()


#### Create unique index in table news for columns ticker and date

In [2]:
import sqlite3

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
try:
    c.execute('CREATE UNIQUE INDEX idx_unique_news_ticker_date ON news(ticker,date)')
except Exception as e:
    print("create unique index for news table: ",e)    

c.close()
conn.commit()
conn.close()

#### Total number of news in news table

In [ ]:
import pandas as pd
import sqlite3, os
from datetime import datetime, timedelta

sources = {}
conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute("SELECT * FROM news where date >= '2022-03-01' and date <= '2024-11-01' and ticker in ('aapl')")
data = c.fetchall()
c.close()
conn.close()
import json
total_items = 0
skipped_items = 0
for row in data:
    df = json.loads(row[2])
    if 'items' in df:
        if df['items'] == '0':
            continue
        if 'feed' in df:
            if len(df['feed']) == 0:
                continue
            for item in df['feed']:
                if 'source' in item:
                    if len(item['source']) == 0:
                        continue
                if item['source'] not in sources:
                    sources[item['source']] = 1
                else:
                    sources[item['source']] += 1
        total_items += int(df['items'])
    else:
        skipped_items += 1
print(total_items)
print(skipped_items)
print(sources)

#### Get news from alphavantage API

In [30]:
from datetime import datetime
import requests
import json
import time
import os
from datetime import timedelta
import sqlite3

START_DATE_STR = "01032022"
LIMIT = 1000
start_time = time.time()
# GET https://www.alphavantage.co/query?apikey=NS9LMM2DJBDQC1W7&function=NEWS_SENTIMENT&sort=EARLIEST&tickers=MSFT&time_from=20220302T0130&time_to=20220303T0130&limit=1000

# Set the API endpoint and parameters
ENDPOINT = "https://www.alphavantage.co/query"
PREMIUM_APIKEY = "**************"
ALPHA_FUNCTION = "NEWS_SENTIMENT"

LOG_FILE = "./news.log"
# Truncate the log file
with open(LOG_FILE, "w") as file:
    file.write("")
    file.close()

conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute('SELECT DISTINCT(ticker) FROM stocks ORDER BY ticker')
tickers_log = c.fetchall()
conn.close()

total_counter = 0
ticker_counter = 0
local_ticker_counter = 0

for ticker in tickers_log:
    ticker = ticker[0]
    os.makedirs('./news/data/' + ticker, exist_ok=True) 

    ticker_counter += 1
    local_ticker_counter = 0

    # Get the current date
    current_date = datetime.now().date()

    # Loop through each day from a specific start date to the current date
    start_date = datetime.strptime(START_DATE_STR, "%d%m%Y").date()
    delta = timedelta(days=1)

    while start_date < current_date:
        local_ticker_counter += 1
        total_counter += 1
        # Format the date as YYYY-MM-DD
        date_from_str = start_date.strftime("%Y%m%d")
        date_to_str = (start_date + delta).strftime("%Y%m%d")

        # Set the time range for the API request
        time_from = f"{date_from_str}T0000"
        time_to = f"{date_to_str}T0000"

        # Continue if the data has already been retrieved
        model_name = f"./news/data/{ticker}/{date_from_str}.json"
        if os.path.exists(model_name):
            # with open(LOG_FILE, "a") as file:
                # file.write(f"{ticker_counter:<4} {local_ticker_counter:<4} {total_counter:<7} Data for {date_from_str} already exists\n")
                # file.close()
            start_date += delta
            continue

        time.sleep(2)
        # Construct the API request URL
        url = f"{ENDPOINT}?apikey={PREMIUM_APIKEY}&function={ALPHA_FUNCTION}&sort=EARLIEST&tickers={ticker}&time_from={time_from}&time_to={time_to}&limit={LIMIT}"

        # Send the API request
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception if the request was not successful
            if response.status_code == 200:
                # Parse the JSON response
                data = response.json()

                # Save the JSON data to a file
                # filename = f"./news/data/{ticker}/{date_from_str}.json"
                with open(model_name, "w") as file:
                    json.dump(data, file)
                    file.close()

                with open(LOG_FILE, "a") as file:
                    file.write(f"{ticker_counter:<4} {local_ticker_counter:<4} {total_counter:<7} Saved data for {date_from_str} to {model_name}\n")
                    file.close()
            else:
                print(f"Failed to retrieve data for {date_from_str}")

            # Move to the next day
            start_date += delta
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            with open(LOG_FILE, "a") as file:
                file.write(f"An error occurred: {e}. Perform request again\n")
                file.close()
            # Handle the error here        

# Record the end time
end_time = time.time()

# Calculate the execution time
execution_time = end_time - start_time
with open(LOG_FILE, "a") as file:
    file.write(f"Execution time: {execution_time} seconds\n")

    

In [ ]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=CE&apikey=UQCN3VITQV0UWTZD'
r = requests.get(url)
data = r.json()

print(data)

#### Get news for specific ticker and date

In [ ]:
from datetime import datetime
import requests
import json
import time
import os
from datetime import timedelta
import sqlite3

START_DATE_STR = "01122021"
LIMIT = 200
# GET https://www.alphavantage.co/query?apikey=NS9LMM2DJBDQC1W7&function=NEWS_SENTIMENT&sort=EARLIEST&tickers=MSFT&time_from=20220302T0130&time_to=20220303T0130&limit=1000

# Set the API endpoint and parameters
ENDPOINT = "https://www.alphavantage.co/query"
PREMIUM_APIKEY = "**************"
ALPHA_FUNCTION = "NEWS_SENTIMENT"

LOG_FILE = "./news.log"
# Truncate the log file
with open(LOG_FILE, "w") as file:
    file.write("")
    file.close()
start_date = datetime.strptime(START_DATE_STR, "%d%m%Y").date()
delta = timedelta(days=1)
date_from_str = start_date.strftime("%Y%m%d")
date_to_str = (start_date + delta).strftime("%Y%m%d")

# Set the time range for the API request
time_from = f"{date_from_str}T0000"
time_to = f"{date_to_str}T0000"
ticker = 'nvda'



url = f"{ENDPOINT}?apikey={PREMIUM_APIKEY}&function={ALPHA_FUNCTION}&sort=EARLIEST&tickers={ticker}&time_from={time_from}&time_to={time_to}&limit={LIMIT}"

# Send the API request
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise an exception if the request was not successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        print(data)
    else:
        print(f"Failed to retrieve data for {date_from_str}")

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")



#### Get data from eodhd

In [ ]:
from eodhd import APIClient

api = APIClient(api_key="674210e833d2f0.82915138")
print(api.get_sentiment("aapl.us", from_date="2014-01-01", to_date="2018-01-01"))
# print(api.financial_news(s="aapl", from_date="2018-01-01", to_date="2018-02-22"))

# import requests

# url = f'https://eodhd.com/api/sentiments?s=btc-usd.cc,aapl.us&from=2022-01-01&to=2022-04-22&api_token=674210e833d2f0.82915138&fmt=json'
# data = requests.get(url).json()

# print(data)


#### Polygon

In [ ]:
jPJVdNzmDVFgubp7IHptXPjBqko0o8nx



TODO: make sctipt to clean up news for companies that are no longer in S&P 500

### Indicators

#### Global variables

In [35]:
indicators = ['200_MA', 'AARON', 'MACD', 'OBV', 'RSI', 'ADX', 'AD']

#### Reset indicators table

In [28]:
import sqlite3

conn = sqlite3.connect('data/stocks.db', timeout=30)
try:
	execute_with_retry(conn, 'DROP INDEX IF EXISTS idx_unique_indicator_ticker_date')
	execute_with_retry(conn, "DROP TABLE IF EXISTS indicator")
	execute_with_retry(conn, "VACUUM")
	execute_with_retry(conn, "CREATE TABLE indicator (date text, ticker text, indicator text, value real) STRICT")
	execute_with_retry(conn, 'CREATE UNIQUE INDEX idx_unique_indicator_ticker_date_indicator ON indicator(ticker,date,indicator)')
finally:
	if conn:
		conn.close()

#### Calculate indicators for the company

In [31]:
import pandas as pd
import numpy as np

def get_indicators(ticker, data_dir):
    try:
        df = pd.read_csv(data_dir+f'/{ticker}.csv')
    except Exception as e:
        print("Error reading file: ", e)
        return None
    for indicator in indicators:
        if indicator == '200_MA':
            df[indicator] = df['Close'].rolling(window=200).mean()
            df[indicator] = df[indicator].fillna(0)
            continue
        if indicator == 'AARON':
            # calculate the Aroon Oscillator for the dataset for the last 25 days
            df['Aroon_Up'] = df['High'].rolling(window=25).apply(lambda x: (25 - np.argmax(x)) / 25)
            df['Aroon_Down'] = df['Low'].rolling(window=25).apply(lambda x: (25 - np.argmin(x)) / 25)
            df[indicator] = df['Aroon_Up'] - df['Aroon_Down']
            continue
        if indicator == 'MACD':
            # calculate the MACD for the dataset
            df['26_EMA'] = df['Close'].ewm(span=26).mean()
            df['12_EMA'] = df['Close'].ewm(span=12).mean()
            df[indicator] = df['12_EMA'] - df['26_EMA']
            continue
        if indicator == 'OBV':
            # calculate the On Balance Volume for the dataset
            df[indicator] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], np.where(df['Close'] < df['Close'].shift(1), -df['Volume'], 0)).cumsum()
            continue
        if indicator == 'RSI':
            # calculate the Relative Strength Index for the dataset
            delta = df['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
            rs = gain / loss
            df[indicator] = 100 - (100 / (1 + rs))
            continue
        if indicator == 'ADX':
            # calculate the Average Directional Index for the dataset
            df['High-Low'] = df['High'] - df['Low']
            df['High-PreviousClose'] = np.abs(df['High'] - df['Close'].shift(1))
            df['Low-PreviousClose'] = np.abs(df['Low'] - df['Close'].shift(1))
            df['TR'] = df[['High-Low', 'High-PreviousClose', 'Low-PreviousClose']].max(axis=1)
            df['+DM'] = np.where((df['High'] - df['High'].shift(1)) > (df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
            df['-DM'] = np.where((df['Low'].shift(1) - df['Low']) > (df['High'] - df['High'].shift(1)), df['Low'].shift(1) - df['Low'], 0)
            df['+DM1'] = np.where(df['+DM'] > df['-DM'], df['+DM'], 0)
            df['-DM1'] = np.where(df['+DM'] < df['-DM'], df['-DM'], 0)
            df['TR14'] = df['TR'].rolling(window=14).sum()
            df['+DM14'] = df['+DM1'].rolling(window=14).sum()
            df['-DM14'] = df['-DM1'].rolling(window=14).sum()
            df['+DI14'] = (df['+DM14'] / df['TR14']) * 100
            df['-DI14'] = (df['-DM14'] / df['TR14']) * 100
            df['DI_diff'] = np.abs(df['+DI14'] - df['-DI14'])
            df['DI_sum'] = df['+DI14'] + df['-DI14']
            df[indicator] = (df['DI_diff'] / df['DI_sum']) * 100
            continue
        if indicator == 'AD':
            # calculate the Accumulation/Distribution for the dataset
            df['MFM'] = ((df['Close'] - df['Low']) - (df['High'] - df['Close'])) / (df['High'] - df['Low'])
            df['MFM'].fillna(0)
            df['MFV'] = df['MFM'] * df['Volume']
            df[indicator] = df['MFV'].cumsum()
            continue
        else:
            df[indicator] = df[indicator].fillna(0)
    return df


#### Write indicators to indicators table for the comapny

In [ ]:
import sqlite3, os
from datetime import datetime
try:
    conn = sqlite3.connect('data/stocks.db')
    c = conn.cursor()
    c.execute('SELECT DISTINCT(ticker) FROM stocks ORDER BY ticker')
    tickers_log = c.fetchall()
    c.close()
    for ticker in tickers_log:
        ticker = ticker[0]
        print('Processing ticker {}'.format(ticker))
        df = get_indicators(ticker, 'data/20241209')
        if df is None:
            print('No data for ticker {}'.format(ticker))
            continue
        for index, row in df.iterrows():
            date = row['Date'][:10]
            for indicator in indicators:
                value = row[indicator]
                if value is None or value == 0.0:
                    continue
                execute_with_retry(
                                    conn,
                                    "INSERT OR REPLACE INTO indicator (date, ticker, indicator, value) VALUES (?, ?, ?, ?)", 
                                    (date, ticker, indicator, value))
finally:
	if conn:
		conn.close()                
            


### Model


#### Generate train data

In [4]:
from sklearn.preprocessing import MinMaxScaler

# Generate training data with a sliding window of size seq_len.
# Return X_train and Y_train
def generate_train_data(data,  seq_len=10,up_front_prediction_days=7, tech_indicator="200_MA", indicators=[], no_news=False, no_indicators=False):
    features = ['volume','close']
    if not no_news:
        features = ['volume','close','avg_sentiment']
    if not no_indicators:
        features.append(tech_indicator)
    target = 'close'

    # Prepare the dataset
    X = data[features]
    Y = data[target]

    scaler = MinMaxScaler(feature_range=(0, 1))

    X_scaled = scaler.fit_transform(X)
    Y_scaled = scaler.fit_transform(pd.DataFrame(Y))  # Convert Y to DataFrame

    X = X_scaled
    Y = Y_scaled

    split = int(0.8 * len(X_scaled))
    x_train, x_test = X_scaled[:split], X_scaled[split:]
    y_train, y_test = Y_scaled[:split], Y_scaled[split:]

    x_train, x_test = pd.DataFrame(x_train), pd.DataFrame(x_test)
    y_train, y_test = pd.DataFrame(y_train), pd.DataFrame(y_test)

    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    # Calculate the 200-day moving average
    # if self.tech_indicator == '200_MA':
        # X['200_MA'] = X['Close'].rolling(window=200).mean()
    # Based on them construct technical indicators: 
    # OBV, A/D, Average directional index, Aroon oscillator, MACD, RSI, Stochastic oscillator, SMA
        

    # TODO: Add VIX (Volatility Index) to the dataset
    
    for i in range((len(x_train)//seq_len)*seq_len - seq_len - up_front_prediction_days):
        x = np.array(x_train.iloc[i: i + seq_len])
        y = np.array([y_train.iloc[i + seq_len + up_front_prediction_days]], np.float64)
        X_train.append(x)
        Y_train.append(y)
    X_train = np.array(X_train)
    Y_train = np.array(Y_train)

    for i in range((len(x_test)//seq_len)*seq_len - seq_len - up_front_prediction_days):
        x = np.array(x_test.iloc[i: i + seq_len])
        y = np.array(y_test.iloc[i + seq_len + up_front_prediction_days], np.float64)
        X_test.append(x)
        Y_test.append(y)
    X_test = np.array(X_test)
    Y_test = np.array(Y_test)

    return X_train, Y_train, X_test, Y_test


#### Generate inference data

In [8]:
from sklearn.preprocessing import MinMaxScaler


def stock_data_to_df(ticker,date_to_predict, seq_len=35, up_front_prediction_days=5, tech_indicator="200_MA",  no_news=False, no_indicators=False):
    conn = sqlite3.connect('data/stocks.db')
    c = conn.cursor()
    c.execute('SELECT date, close, volume FROM stocks WHERE ticker = ? and date <= ? ORDER BY date DESC LIMIT ?', (ticker,date_to_predict.strftime("%Y-%m-%d"),seq_len + up_front_prediction_days))
    prices = c.fetchall()
    data = pd.DataFrame(prices, columns=['date', 'close', 'volume'])
    c.close()
    # reverse data
    data = data.iloc[::-1]
    min_date = data.iloc[0]['date']
    max_date = data.iloc[-1]['date']
    if not no_news: 
        c = conn.cursor()
        c.execute('SELECT date, avg_sentiment FROM news WHERE ticker = ? and date >= ? and date <= ?', (ticker,min_date,max_date ))
        news_sentiment = c.fetchall()
        c.close()
        data = pd.merge(data, pd.DataFrame(news_sentiment, columns=['date', 'avg_sentiment']), on='date', how='left')
        # Fill NaN values in avg_sentiment with 0
        data['avg_sentiment'] = data['avg_sentiment'].fillna(0)

    if not no_indicators:
        c = conn.cursor()
        c.execute('SELECT date, value FROM indicator WHERE ticker = ? and date <= ? and indicator = ? ORDER BY date DESC LIMIT ?', (ticker,date_to_predict.strftime("%Y-%m-%d"),tech_indicator,seq_len + up_front_prediction_days ))
        tech_indicators = c.fetchall()
        c.close()
        data = pd.merge(data, pd.DataFrame(tech_indicators, columns=['date', tech_indicator]), on='date', how='left')

    predicted_stock_price = data.iloc[-1]['close']
    data = data.iloc[:-up_front_prediction_days]
    return data, predicted_stock_price


# Generate training data with a sliding window of size seq_len.
# Return X_train and Y_train
def generate_inference_data(data, tech_indicator="200_MA", no_news=False, no_indicators=False):
    features = ['volume','close']
    if not no_news:
        features = ['volume','close','avg_sentiment']
    if not no_indicators:
        features.append(tech_indicator)

    # Prepare the dataset
    X = data[features]

    scaler = MinMaxScaler(feature_range=(0, 1))
    X_scaled = scaler.fit_transform(X)
    X_reshaped = X_scaled.reshape(1, X_scaled.shape[0], X_scaled.shape[1])


    return X_reshaped, scaler


#### Build model

In [3]:
from tensorflow import keras
from tensorflow.keras.models import Sequential

def build_model(seq_len, amount_of_params, optimizer, loss_function, activation, amount_of_neurons):
    model = Sequential()
    # Adding the first LSTM layer
    model.add(
        keras.layers.Input(shape=(seq_len, amount_of_params)),
        keras.layers.LSTM(amount_of_neurons, return_sequences=True,
                        activation=activation))
    model.add(keras.layers.LSTM(64, return_sequences=True,  activation=activation))
    model.add(keras.layers.LSTM(40, activation=activation))
    model.add(keras.layers.Dense(1, activation=activation))
    # Compiling the RNN
    model.compile(optimizer=optimizer, loss=loss_function)
    return model

#### Log file

In [2]:
LOG_FILE = './logs/model.log'
def output_in_logfile(message):
    with open(LOG_FILE, "a") as file:
        file.write(message+'\n')
        file.close()

#truncate log file
with open(LOG_FILE, "w") as file:
    file.write("")
    file.close()


#### Train model

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from tensorflow.keras.models import Sequential
from datetime import timedelta
import sqlite3
import os


SEQ_LEN = 35
AMOUNT_OF_PARAMS = 4
UP_FRONT_PREDICTION_DAYS = 5
AMOUNT_OF_NEURONS = 128
# OPTIMIZERS = ['adam', 'sgd', 'rmsprop', 'adagrad', 'adamax', 'nadam']
OPTIMIZERS = ['adam', 'rmsprop', 'nadam']
OPTIMIZER = OPTIMIZERS[2]
LOSS_FUNCTIONS = ['mean_squared_error', 'huber', 'log_cosh']
LOSS_FUNCTION = LOSS_FUNCTIONS[1]
# ACTIVATIONS = ['relu', 'tanh','leaky_relu','elu','selu']
ACTIVATIONS = ['relu', 'tanh','leaky_relu','elu']
ACTIVATION = ACTIVATIONS[1]
COMPANIES_CSV_PATH = './news/constituents.csv'

# STOCH, SO - ?
TECH_INDICATORS = ['RSI', 'OBV', 'ADX', 'MACD', 'AD', 'AARON','200_MA']
TECH_INDICATOR = TECH_INDICATORS[0]
NUM_THREADS = 4
WITHOUT_NEWS = True
WITHOUT_INDICATORS = False
EPOCHS = 20


conn = sqlite3.connect('data/stocks.db')
c = conn.cursor()
c.execute('SELECT DISTINCT(ticker) FROM stocks ORDER BY ticker')
tickers_log = c.fetchall()
c.close()

# date one year ago
DATE_ONE_YEAR_AGO = (datetime.now() - timedelta(days=365)).strftime("%Y-%m-%d")

for ticker in tickers_log:
    ticker = ticker[0]
    if ticker != 'nvda':
        continue

    c = conn.cursor()
    c.execute('SELECT date, close, volume FROM stocks WHERE ticker = ? and date >= "2022-03-01" and date <= ? ORDER BY date', (ticker,DATE_ONE_YEAR_AGO))
    prices = c.fetchall()
    c.close()
    print(f'Prices: {len(prices)}')
    c = conn.cursor()
    c.execute('SELECT date, avg_sentiment FROM news WHERE ticker = ? and date <= ? ORDER BY date', (ticker,DATE_ONE_YEAR_AGO))
    news_sentiment = c.fetchall()
    c.close()

    # [('200_MA',), ('AARON',), ('AD',), ('ADX',), ('MACD',), ('OBV',), ('RSI',)]
    query = f'SELECT date, value as "{TECH_INDICATOR}" FROM indicator WHERE ticker = ? and date >= "2022-03-01" and date <= ? and indicator = ? ORDER BY date'
    c = conn.cursor()
    c.execute(query, (ticker, DATE_ONE_YEAR_AGO, TECH_INDICATOR))
    tech_indicators = c.fetchall()
    c.close()


    combined_data = pd.DataFrame(prices, columns=['date', 'close', 'volume'])
    if not WITHOUT_NEWS:
        combined_data = pd.merge(combined_data, pd.DataFrame(news_sentiment, columns=['date', 'avg_sentiment']), on='date', how='left')
        # Fill NaN values in avg_sentiment with 0
        combined_data['avg_sentiment'] = combined_data['avg_sentiment'].fillna(0)
    else:
        AMOUNT_OF_PARAMS -= 1
    if not WITHOUT_INDICATORS:
        combined_data = pd.merge(combined_data, pd.DataFrame(tech_indicators, columns=['date', TECH_INDICATOR]), on='date', how='left')
    else:
        AMOUNT_OF_PARAMS -= 1 

    len_before = len(combined_data)
    # remove rows with NaN values
    combined_data = combined_data.dropna()
    len_after = len(combined_data)
    print(f'Removed {len_before - len_after} rows with NaN values')

    X_train, Y_train, X_test, Y_test = generate_train_data(combined_data, SEQ_LEN, UP_FRONT_PREDICTION_DAYS, tech_indicator=TECH_INDICATOR, indicators=[TECH_INDICATOR], no_news=WITHOUT_NEWS, no_indicators=WITHOUT_INDICATORS)
    model = build_model(SEQ_LEN, AMOUNT_OF_PARAMS, OPTIMIZER, LOSS_FUNCTION, ACTIVATION, AMOUNT_OF_NEURONS)
    try:
        model.fit(tf.convert_to_tensor(X_train,dtype=tf.float32), 
                tf.convert_to_tensor(Y_train,dtype=tf.float32), 
                epochs=EPOCHS,  
                batch_size=128, 
                validation_batch_size = 64, 
                verbose='0')
        loss = model.evaluate(tf.convert_to_tensor(X_test,dtype=tf.float32), 
                            tf.convert_to_tensor(Y_test,dtype=tf.float32), 
                            verbose='0', 
                            batch_size=128)
        keras.backend.clear_session()
        tf.compat.v1.reset_default_graph()
    except Exception as e:
        output_in_logfile(f"Error training {ticker}: {e}")
        continue    

    # check if loss is float
    loss_evaluation = "good"
    theshold = 0.005
    loss_value = 0.0
    if isinstance(loss, float):
        if loss > theshold:
            loss_evaluation = "bad"
            print(f'Loss for {ticker} is {loss}')
            del model
            continue
        loss_value = round(float(loss), 10)
        theshold = round(theshold, 10)
    # Get the current date and time
    now = datetime.now()

    # Format the date and time
    formatted_date_time = now.strftime("%Y-%m-%d_%H:%M:%S")
    indicator_str = TECH_INDICATOR
    news_str = "with_news"
    if WITHOUT_INDICATORS:
        indicator_str = "no_indicators"
    if WITHOUT_NEWS:    
        news_str = "no_news"
    else:
        indicator_str = TECH_INDICATOR
    os.makedirs(f'./models/LSTM/{ticker.lower()}', exist_ok=True)
    model.save(f'./models/LSTM/{ticker.lower()}/model-{formatted_date_time}-{AMOUNT_OF_PARAMS}-{SEQ_LEN}-{AMOUNT_OF_NEURONS}-{OPTIMIZER}-{ACTIVATION}-{indicator_str}-{LOSS_FUNCTION}-{loss_evaluation}-{loss_value}-{theshold}-{news_str}-pending.keras')    


conn.close()



#### Inference

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
from datetime import timedelta
import sqlite3
import os
from tensorflow.keras.models import load_model
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Pool

UP_FRONT_PREDICTION_DAYS = 5
path_predictions_db = './data/predictions.db'

def insert_prediction_into_db(ticker, date, predicted_stock_price, actual_stock_price, filename, with_news, indicator, optimizer, activation, loss_function, loss):
    conn = sqlite3.connect(path_predictions_db, timeout=30)
    execute_with_retry(conn, 'INSERT OR REPLACE INTO predictions VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)', (ticker, date, predicted_stock_price, actual_stock_price, filename, with_news, indicator, optimizer, activation, loss_function, loss))
    conn.commit()
    conn.close()

# def predict_stock_price(data, filename, ticker, date, actual_stock_price, tech_indicator, optimizer, activation, loss_function, no_news, no_indicators):
def predict_stock_price(data_tuple):
    data, filename, ticker, date, actual_stock_price, tech_indicator, optimizer, activation, loss_function, no_news, no_indicators = data_tuple
    model = load_model(f'./bigdata/one/models/LSTM/{ticker.lower()}/{filename}')
    try:    
        X, scaler = generate_inference_data(data, tech_indicator=tech_indicator, no_news=no_news, no_indicators=no_indicators)
    except Exception as e:
        print(f"Error generating inference data for {filename}: {e}")
        return 0
    # print(f'X: {len(X)}')
    # print(f'X shape: {X.shape}')
    # print(f'Model input shape: {model.input_shape}')
    try:
        predicted_stock_price = model.predict(X, verbose=0)
    except Exception as e:
        print(f"Error predicting stock price for {filename}: {e}")
        return 0
    dummy_array = np.zeros((1,X.shape[2]))
    dummy_array[0][1] = predicted_stock_price[0][0]
    predicted_stock_price = scaler.inverse_transform(dummy_array)[0][1]
    with_news = 1 if no_news == False else 0
    date_to_predict = date.strftime("%Y-%m-%d")
    insert_prediction_into_db(ticker, date_to_predict, predicted_stock_price, actual_stock_price, filename, with_news, tech_indicator, optimizer, activation, loss_function, loss)
    del model
    return predicted_stock_price



path_stocks_db = 'data/stocks.db'
conn_stocks = sqlite3.connect(path_stocks_db, timeout=30)

thursdays = get_dates_of_day_of_week_in_range('2023-11-01', '2024-11-01', 3)
traded_thursdays = filter_dates_not_in_stock_table(thursdays)


path_bigdata_logs_db = 'bigdata/one/logs.db'
conn = sqlite3.connect(path_bigdata_logs_db, timeout=30)
c = conn.cursor()
c.execute('SELECT ticker, filename, amount_of_params, seq_len, amount_of_neurons_l1, amount_of_neurons_l2, amount_of_neurons_l3, optimizer, activation, indicator, loss_function, loss_evaluation, loss, threshold, with_news FROM log_models WHERE ticker = "acgl" AND loss < 0.005')
res = c.fetchall()
c.close()
conn.close()
print(f'Res: {len(res)}')



for row in res:
    ticker = row[0]
    model_name = row[1]
    amount_of_params = row[2]
    seq_len = row[3]
    amount_of_neurons_l1 = row[4]
    amount_of_neurons_l2 = row[5]
    amount_of_neurons_l3 = row[6]
    optimizer = row[7]
    activation = row[8]
    indicator = row[9]
    loss_function = row[10]
    loss_evaluation = row[11]
    loss = row[12]
    threshold = row[13]
    with_news = row[14]

    no_news = with_news == 0
    no_indicators = indicator == "no_indicator" or indicator == ""
    
    data_batch = []
    for thursday in traded_thursdays:
        date_to_predict = thursday
        
        conn = sqlite3.connect(path_predictions_db, timeout=30)
        c = conn.cursor()
        c.execute('SELECT 1 FROM predictions WHERE ticker = ? AND filename = ? AND date = ? ', (ticker, model_name, date_to_predict.strftime("%Y-%m-%d")))
        exists = c.fetchall()
        c.close()
        conn.close()
        if len(exists) > 0:
            print(f'Prediction for {ticker} {model_name} {date_to_predict} already exists')
            continue

        data, actual_stock_price = stock_data_to_df(
            ticker, 
            date_to_predict, 
            seq_len = seq_len, 
            up_front_prediction_days=UP_FRONT_PREDICTION_DAYS, 
            tech_indicator=indicator, 
            no_news=no_news, 
            no_indicators=no_indicators)
        data_batch.append((data, model_name, ticker, date_to_predict, actual_stock_price, indicator, optimizer, activation, loss_function, no_news, no_indicators))
    # with Pool(processes=4) as pool:  # Adjust number of processes
        # _ = pool.map(predict_stock_price, data_batch)
    with ThreadPoolExecutor(max_workers=2) as executor:
        _ = list(executor.map(predict_stock_price, data_batch))    
    # print(f'Results: {results}')




#### Create predictions table

In [71]:
import sqlite3
path_predictions_db = './data/predictions.db'
conn = sqlite3.connect(path_predictions_db, timeout=30)
c = conn.cursor()
try:
    c.execute('DROP INDEX IF EXISTS idx_predictions_ticker_date_filename')
    c.execute('DROP TABLE IF EXISTS predictions')
    c.execute('VACUUM')
    c.execute('CREATE TABLE EXISTS predictions (ticker text, date text, predicted_stock_price real, actual_stock_price real, filename text, with_news integer, indicator text, optimizer text, activation text, loss_function text, loss real) STRICT')
    c.execute('CREATE INDEX EXISTS idx_predictions_ticker_date_filename ON predictions(ticker, filename, date)')
except Exception as e:
    print(f'Error creating predictions table: {e}')
conn.commit()
conn.close()

#### Precision of the models

In [ ]:
import sqlite3, os
import numpy as np
from sklearn.metrics import mean_absolute_error

# Assuming y_true are the true labels and y_pred are the predicted labels

conn = sqlite3.connect('data/predictions.db', timeout=30)
c = conn.cursor()
c.execute('SELECT DISTINCT(ticker) FROM predictions')
res = c.fetchall()
ticker_list = [row[0] for row in res]
c.close()
precision_list  = []
for ticker in ticker_list:
    c = conn.cursor()
    c.execute('SELECT predicted_stock_price, actual_stock_price  FROM predictions where with_news = 1 and indicator != "" and predicted_stock_price is not null and actual_stock_price is not null and ticker = ?', (ticker,))
    res = c.fetchall()
    c.close()
    counter += len(res)
    # print(f'Counter: {counter}')
    counter = 0
    # average precision
    y_true = []
    y_pred = [] 
    ticker_list = []
    precision_sum = 0
    for row in res:
        y_true.append(row[1])
        y_pred.append(row[0])
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    error = (np.abs(y_true - y_pred)/y_true) * 100
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    precision_list.append((mape, ticker))
    # print(f'Ticker: {ticker}; MAPE: {mape:.2f}%')
conn.close()
# remove nan from precision_list
precision_list = [x for x in precision_list if not np.isnan(x[0])]
precision_list.sort(key=lambda x: x[0])
# median of precision_list
median_value = np.median([x[0] for x in precision_list])
for row in precision_list:
    print(f'Ticker: {row[1]}; MAPE: {row[0]:.2f}%')
print(f'Median MAPE: {median_value:.2f}%')